Import Pyomo and course code.

In [1]:
from pyomo.environ import *
from brooks import *


Define data.

In [2]:
L = ['peach', 'raspberry']
profit = {'peach': 8, 'raspberry': 12}
sugar_req = {'peach': 4, 'raspberry': 3}
juice_req = {'peach': 1, 'raspberry': 2}
demand = {'peach': 4, 'raspberry': 2}
juice_avail = 5
sugar_avail =12 

Declare a model object.

In [3]:
model = Model()

Declare nonnegative variables for peach and raspberry lemonade.

In [4]:
if hasattr(model,'x'):
    model.delete(model.x)
model.x = Var(L, domain=NonNegativeReals)

Define the objective function.

In [5]:
if hasattr(model, 'profit_objective'):
    model.delete(model.profit_objective)
model.profit_objective = Objective(expr = sum(profit[i]*model.x[i] for i in L), sense=maximize)

Specify the juice constraint. 

In [6]:
if hasattr(model, 'juice_constraint'):
    model.delete(model.juice_constraint)
model.juice_constraint = Constraint(expr = sum(juice_req[i]*model.x[i] for i in L) <= juice_avail)

Specify the sugar constraint.

In [7]:
if hasattr(model, 'sugar_constraint'):
    model.delete(model.sugar_constraint)
model.sugar_constraint = Constraint(expr = sum(sugar_req[i]*model.x[i] for i in L) <= sugar_avail)

In [8]:
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'peach', 'raspberry'}

1 Var Declarations
    x : Size=2, Index=x_index
        Key       : Lower : Value : Upper : Fixed : Stale : Domain
            peach :     0 :  None :  None : False :  True : NonNegativeReals
        raspberry :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    profit_objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[peach] + 12*x[raspberry]

2 Constraint Declarations
    juice_constraint : Size=1, Index=None, Active=True
        Key  : Lower : Body                      : Upper : Active
        None :  -Inf : x[peach] + 2*x[raspberry] :   5.0 :   True
    sugar_constraint : Size=1, Index=None, Active=True
        Key  : Lower : Body                        : Upper : Active
        None :  -Inf 

Add the demand constraints as a set of constraints.  

In [9]:
if hasattr(model, 'demand_constraint'):
    model.delete(model.demand_constraint)
    
model.demand_constraint = ConstraintList()
for i in L:
    model.demand_constraint.add(model.x[i] <= demand[i])

Specify solver and solve.


In [10]:

solver = SolverFactory('glpk')
status = solver.solve(model)

Get status.

In [11]:
print('Status = %s' % status)

Status = 
Problem: 
- Name: unknown
  Lower bound: 33.6
  Upper bound: 33.6
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 3
  Number of nonzeros: 7
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.017319917678833008
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



Get solution and objective function value.

In [12]:
for i in L:
    print('%s = %f' % (model.x[i], value(model.x[i])))
print("Objective = %f" % value(model.profit_objective))


x[peach] = 1.800000
x[raspberry] = 1.600000
Objective = 33.600000


The optimal solution is to produce 1.8 gallons of peach lemonade and 1.5 gallons of raspberry lemonade for a profit of \$33.60.